In [1]:
#Alternative approach
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=2599fc645ef1a5de92494ed8b159a649d621ebeea47113ff2674eaf8cfadfe7e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import os
import sys
import pyspark
import json
import pandas as pd
import gzip
import numpy as np
import numpy as np
import re

In [4]:
#extract data by start and end
def extraxtString(string,startString,endString):
    startingPoint=0
    lengthString=len(string)
    myList = []
    continueYes = 1
    while(continueYes == 1 and startingPoint < lengthString):
        temp1=string.find(startString,startingPoint)
        if(temp1 != -1):
            temp1 = temp1+len(startString)
            temp2 = string.find(endString,temp1)
            if(temp2 != -1):
                #middleString=string[temp1:temp2]
                middleString=string[(temp1-len(startString)):temp2]
                myList.append(middleString)
                #startingPoint=temp2+len(endString)
                startingPoint=temp2
            else:
                continueYes=0
        else:
            continueYes=0
    return myList

In [5]:
import findspark
findspark.init()
from pyspark.sql.functions import from_json, udf, col, lower, regexp_replace, split, when
from pyspark.sql.functions import rand #additional library for optimization
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType
import random
import json
import re
import string
import time


In [6]:
!pip install delta

  Preparing metadata (setup.py) ... done
  Created wheel for delta: filename=delta-0.4.2-py3-none-any.whl size=2914 sha256=2dd682ddc2b31017b28cb7faa9286f032e94709ab696e01283c625757942ba29
  Stored in directory: /root/.cache/pip/wheels/a8/86/24/a486f14769cf86a2a9ce6b589a82b7414b14657c6fd515dc75
Successfully built delta


In [7]:
!pip install matplotlib-venn

In [1]:
!pip install delta-spark

In [2]:
import findspark
findspark.init()
from pyspark.sql.functions import from_json, udf, col, lower, regexp_replace, split, when
from pyspark.sql.functions import rand #additional library for optimization
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType
import random
import json
import re
import string
import time

from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

In [3]:
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.base import Transformer
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.sql.functions import when

In [5]:
#Added last two .config to allocate memory and ensure 4 cores to be utilized
#Set memory per exector to 2g to be able to handle the 5GB file
builder = SparkSession.builder.appName("amazon") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.cores", "3") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory", "2g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [6]:
#Create a dataframe from the txt file by reading it as json for formating
df = spark.read.json("/content/drive/MyDrive/GoogleColab/Video_Games_5.json.gz")

In [7]:
df.show()

+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|     reviewerName|style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|0700026657| NULL|    5.0|This game is a bi...|10 17, 2015|A1HP7NVNPFMA4N|      Ambrosia075| NULL|but when you do i...|    1445040000|    true|NULL|
|0700026657| NULL|    4.0|I played it a whi...|07 27, 2015|A1JGAP0185YJI6|           travis| NULL|But in spite of t...|    1437955200|   false|NULL|
|0700026657| NULL|    3.0|            ok game.|02 23, 2015|A1YJWEXHQBWK2B|Vincent G. Mezera| NULL|         Three Stars|    1424649600|    true|NULL|
|0700026657| NULL|    2.0|found the game a ...|02 20, 2015|A2204E1TH211HT|       Grandma KR| NULL|        

In [8]:
df = df.select("overall","reviewText")
#df.show(truncate=False)

In [9]:
#Create a delta frame from the df and save to the hdfs
df.write.format("delta").mode("overwrite").save("/content/drive/MyDrive/GoogleColab")

In [10]:
#fetch the delta table from hdfs
delta_df = spark.read.format("delta").load("/content/drive/MyDrive/GoogleColab")
delta_df.show(3,truncate=False)

+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|overall|reviewText                                                                                                                                                                                                                                                                                               |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|5.0    |This game is a bit hard to get the hang of, but when you do it's gr

In [11]:
# Convert 'overall' to binary (0 or 1)
delta_df = delta_df.withColumn("overall", when(delta_df["overall"] < 3, 0).otherwise(1))
#delta_df.show(3,truncate=False)

In [12]:
# Define a custom transformer to remove special characters
class RemoveSpecialCharactersTransformer(Transformer):
    def __init__(self, inputCol, outputCol):
        super(RemoveSpecialCharactersTransformer, self).__init__()
        self.inputCol = inputCol
        self.outputCol = outputCol

    def _transform(self, df: DataFrame) -> DataFrame:
#        print(f"Ran _transform with data {df[self.inputCol]}")
        return df.withColumn(self.outputCol, udf(remove_special_characters)(df[self.inputCol]))

# Define the UDF to remove special characters
def remove_special_characters(text):
    print(text, '\n')
    if type(text)==str:
        return re.sub(r'[^A-Za-z0-9\s]', '', text)

    return ""

# Create instances of the custom transformers
remove_special_chars = RemoveSpecialCharactersTransformer(inputCol="reviewText", outputCol="cleanText")


In [13]:
# Tokenize the reviewText
tokenizer = Tokenizer(inputCol="cleanText", outputCol="token")

In [14]:
# Define the StopWordsRemover
stop_words_remover = StopWordsRemover(inputCol="token", outputCol="words")

# Vectorize the filtered words using HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="features")

In [15]:
# Split the data into training and testing sets
train_data, test_data = delta_df.randomSplit([0.7, 0.3])

In [16]:
# Define the Naive Bayes model
nb = NaiveBayes(featuresCol="features", labelCol="overall")

# Create a pipeline with Tokenizer, StopWordsRemover, HashingTF, and Naive Bayes
pipeline = Pipeline(stages=[remove_special_chars,tokenizer, stop_words_remover, hashingTF, nb])


In [17]:
# Train the model
model = pipeline.fit(train_data)

In [18]:
# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="overall", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)



Accuracy: 0.9148268369235721
